<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Описание-и-предобработка-данных" data-toc-modified-id="Описание-и-предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Описание и предобработка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования-и-его-проверка" data-toc-modified-id="Алгоритм-преобразования-и-его-проверка-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования и его проверка</a></span></li><li><span><a href="#Итоговые-выводы" data-toc-modified-id="Итоговые-выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговые выводы</a></span></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 

## Описание проекта

**Заказчик** - страховая компания "Хоть потоп". Перед нами стоит задача защиты личных данных клиентов заказчика. Для этого необходимо разработать некоторый метод преобразования данных, чтобы по нему было сложно восстановить персональную информацию. Перед этим необходимо обосновать корректность предложенного нами метода работы.

Помимо этого также требуется подобрать такой тип защиты, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Исходные данные** - личная информация о клиентах компании. Они включают в себя следующие характеристики: пол, возраст и зарплата застрахованного, количество членов его семьи, а также количество страховых выплат клиенту за последние 5 лет - который и будет являться целевым признаком.

## Описание и предобработка данных

Начнем работу с загрузки данных.

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')
except:
    data = pd.read_csv('insurance.csv')

In [3]:
data.info()
data.head()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Что можно заметить по полученной информации? Датасет состоит из 5000 наблюдений - ни по одной из характеристик пропущенные значения не наблюдаются. Все характеристики имеют численные типы: `float` и `int`. В дополнительной обработке данные не нуждаются. 

Заранее проведем разделение исходных характеристик на две переменные, которые будут отвечать за общие и целевые переменные.

In [4]:
X = data.copy().drop('Страховые выплаты', axis=1)
y = data[['Страховые выплаты']].copy()

## Умножение матриц

Перед нами стоит задача подобрать такой метод для шифровки данных, чтобы качество модели при использовании новых данных не поменялось. Для этого можно попробовать подобрать некоторую матрицу, которая бы преобразовывала данные не влияя на качество модели.

Одним из наиболее оптимальных вариантов будет являться некоторая обратимая матрица. Докажем, что при домножении исходных признаков на такую матрицу качество линейной регрессии не поменяется, путем вывода связи между параметрами регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

По условию нам сказано, что новые признаки получаются путем перемножения исходного набора признаков на обратимую матрицу. Обозначим новую матрицу признаков как $Х'$, а обратимую матрицу как $Р$.

$$
X' = XP,
$$

при этом $Р$ имеет размерность $(NxN)$, где $N$ - число признаков в исходном наборе данных. Такое условие необходимо для возможности перемножения матриц.

Поскольку мы имеем дело с линейной регрессией, выведем формулу для оценки коэффициентов регрессии:

$$
w = (X^T X)^{-1} X^T y,
$$

Выведем формулу новых оценок коэффициентов $w'$ для новых признаков $Х'$:

$$
w' = (X'^T X')^{-1} X'^T y 
$$

Подставим в данное выражение правую часть первого выражения о равенстве $X'$:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y 
$$

Раскроем выражение до первого сокращения:

$$
w' = (P^{T} X^{T} X P)^{-1} (XP)^{T} y = P^{-1} (X^{T} X)^{-1} (P^{T})^{-1} P^{T} X^{T} y
$$

Поскольку $P$ по условию обратимая матрица, то для нее верно $P P^{-1} = P^{-1} P = E$, где $E$ - единичная матрица. На основе данной информации сократим выражение и преобразуем его:

$$
w' = P^{-1} (X^{T} X)^{-1} E X^{T} y = P^{-1} (X^{T} X)^{-1} X^{T} y
$$

Выражение, стоящее после $P^{-1}$ является ничем иным как формулой исходных коэффициентов $w$. Следовательно, исходные коэффициенты $w$ и преобразованные $w'$ связаны следующим образом:

$$
w' = P^{-1} w
$$

Теперь перейдем к выводу предсказаний. Для модели линейной регрессии с исходными переменными предсказания будут вычисляться по следующей формуле:

$$
a = Xw
$$

Предсказания для преобразованных признаков соответственно:

$$
a' = X'w'
$$

Подставим значения полученные значения $X'$ и $w'$ и получим следующее выражение:

$$
a' = XP P^{-1} w = Xw = a
$$

Таким образом, предсказания не поменяются, а значит и качество модели останется неизменным. 

**Что и требовалось доказать!**

## Алгоритм преобразования и его проверка

Наконец, перейдем к завершающему этапу работы - создания алгоритма для шифровки.

**Алгоритм**

*Идея* - перемножение матрицы с исходными признаками на некоторую случайную обратимую матрицу. 

*Этапы работы:*
1. Генерация матрицы
2. Проверка ее на обратимость
3. Перемножения исходных характеристик и матрицы

Доказательства, почему допустимо использование предложенного нами алгоритма были приведены на предыдущем этапе работы. 

Проверим, что предложенный нами алгоритм работает как надо и на практике. Для этого необходимо сравнить качество моделей линейной регрессии, построенных на исходных и зашифрованных данных. 

Начнем с построения модели на исходных данных. 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, random_state=9)

In [6]:
lin_reg_basic = LinearRegression()
lin_reg_basic.fit(X_train, y_train)
r2_basic = r2_score(y_test, lin_reg_basic.predict(X_test))
print(f'Вектор из коэффициентов линейной регрессии для исходных переменных: {lin_reg_basic.coef_}')
print(f'R2 линейной регрессии для исходных переменных: {r2_basic}')

Вектор из коэффициентов линейной регрессии для исходных переменных: [[ 2.26243255e-03  3.54857177e-02  1.74649797e-07 -1.14774564e-02]]
R2 линейной регрессии для исходных переменных: 0.42399220983841


Создадим функцию, которая будет воплощать предложенный нами алгоритм. На вход она получает исходные признаки, а возвращает зашифрованные и обратимую матрицу на тот случай, если в дальнейшем кому-то потребуется расшифровать данные.

In [7]:
def data_convertation(features):
    based = features
    n = based.shape[1]
    np.random.seed(9)
    matrix = np.random.normal(size=(n,n))
    det = np.linalg.det(matrix)
    while det == 0:
        np.random.seed(7072021)
        matrix = np.random.randint(size=(n,n))
        det = np.linalg.det(matrix)
    converting = based @ matrix
    return converting, matrix

In [8]:
X_c, matrix = data_convertation(X)

Выведем зашифрованные переменные и обратимую матрицу. 

In [9]:
display(X_c.head())
display(matrix)

,0,1,2,3
0,-11952.971723,-32157.502617,31478.693751,86290.110477
1,-9162.969928,-24643.428211,24095.886631,66102.308259
2,-5065.264631,-13620.849590,13309.710081,36528.228124
3,-10043.732350,-27028.097927,26488.439723,72553.444857
4,-6292.356111,-16925.190046,16553.155855,45403.304370


array([[ 1.10855471e-03, -2.89544069e-01, -1.11606630e+00,
        -1.28827567e-02],
       [-3.78361464e-01, -4.81135363e-01, -1.51733118e+00,
        -4.90871981e-01],
       [-2.40680579e-01, -6.47947460e-01,  6.35891080e-01,
         1.74011731e+00],
       [ 2.96682218e-01,  7.07503662e-01,  1.82281576e+00,
         4.30769029e-01]])

Как можно заметить, новые данные существенно отличаются от исходных.

Теперь повторим построение модели линейной регрессии на обработанных переменных.

In [10]:
X_train_c, X_test_c, y_train, y_test = train_test_split(X_c, y,
                                                        test_size=0.25, random_state=9)

In [11]:
lin_reg_conv = LinearRegression()
lin_reg_conv.fit(X_train_c, y_train)
r2_conv = r2_score(y_test, lin_reg_conv.predict(X_test_c))
print(f'Вектор из коэффициентов линейной регрессии для зашифрованных переменных: {lin_reg_conv.coef_}')
print(f'R2 линейной регрессии для зашифрованных переменных: {r2_conv}')

Вектор из коэффициентов линейной регрессии для зашифрованных переменных: [[-0.14322628  0.10026219 -0.0285032   0.02793947]]
R2 линейной регрессии для зашифрованных переменных: 0.4239922098384916


Как можно заметить, значения R2 линейной регрессии для зашифрованных переменных практически совпадает с аналогичным показателем для исходных переменных. 

На основе этого мы можем сделать вывод, что предложенный нами алгоритм работает как надо и соответствует всем требованиям заказчика. 

## Итоговые выводы

Перед нами стояла задача создания алгоритма для шифрования личной информации клиентов компании "Хоть потоп". Было необходимо подобрать такой метод, который бы не повлиял на качество линейной модели. 

В качестве алгоритма мы предложили использовать следующее - домножить матрицу с исходными признаками на случайно сгенерированную обратимую матрицу. В качестве подтверждения работоспособности метода мы сделали две проверки: теоретическую и практическую. При теоретической проверке мы математически вывели взаимосвязь между качествами двух моделей - они оказались равны между собой. При практической проверке мы построили две линейные регрессии и сравнили их R2 - существенных различий в их значениях не было обнаружено.

Таком образом, нам удалось подобрать оптимальный метод для шифровки исходных данных, который при этом соответствует всем требованиям заказчика. 